In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

ss = (
    SparkSession.builder.master("local")
    .appName("sparkSTREAM:basicWC")
    .getOrCreate()
)

In [3]:
"""
SETUP: Simple spark streaming

ensure `nc -lk 9999` is running
"""

df_lines = (
    ss.readStream
    .format("socket")
    .option("host", "192.168.93.128")
    .option("port", 9999)
    .load()
)

In [4]:
"""
.show()/print won't work as it's streaming data (not batch data)
.printSchema() works
"""

df_lines.printSchema()

root
 |-- value: string (nullable = true)



In [5]:
"""
Split lines into words.
Convert words into individual spark records using explode (after splitting on the delimiter " ")
.explode() will convert invidual words into separate spark records
"""
import pyspark.sql.functions as f

df_words = df_lines.select(f.explode(f.split(df_lines.value, " ")).alias("word"))


In [6]:
df_word_count = df_words.groupBy("word").count()

In [7]:
"""
PRINT schema on console.

ensure you don't have an open telnet on the same port you're listening to from here.
read data sent from nc command  (linux terminal) and check the output on Jupyter's console
"""

echoOnConsole = (
    df_word_count
    .writeStream
    .outputMode("complete")  # options: append/complete
    .format("console")
    .start()  # Starts the query, and spark will subscribe to stream data
)

"""CHECK JUPYTER CONSOLE"""

In [ ]:
"""
FINALLY: Await termination (before exiting this notebook)
"""

echoOnConsole.awaitTermination()